In [1]:
import pandas as pd
import numpy as np

import os

import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import SnowballStemmer

In [2]:
train_pos_path = 'aclImdb\\train\\pos'
train_neg_path = 'aclImdb\\train\\neg'
test_pos_path = 'aclImdb\\test\\pos'
test_neg_path = 'aclImdb\\test\\neg'

In [3]:
corpus_train = []
corpus_test = []
labels_train = []
labels_test = []

In [4]:
%%time
for fname in os.listdir(train_pos_path):
    with open (os.path.join(train_pos_path, fname), 'r', encoding='utf-8') as f:
        labels_train.append(int(fname.split('_')[1].split('.')[0]))
        corpus_train.append(f.read())

for fname in os.listdir(train_neg_path):
    with open (os.path.join(train_neg_path, fname), 'r', encoding='utf-8') as f:
        labels_train.append(int(fname.split('_')[1].split('.')[0]))
        corpus_train.append(f.read())

for fname in os.listdir(test_pos_path):
    with open (os.path.join(test_pos_path, fname), 'r', encoding='utf-8') as f:
        labels_test.append(int(fname.split('_')[1].split('.')[0]))
        corpus_test.append(f.read())

for fname in os.listdir(test_neg_path):
    with open (os.path.join(test_neg_path, fname), 'r', encoding='utf-8') as f:
        labels_test.append(int(fname.split('_')[1].split('.')[0]))
        corpus_test.append(f.read())

CPU times: total: 1.38 s
Wall time: 2min 40s


In [5]:
data_train = pd.DataFrame({'train_text' : corpus_train, 'train_labels' : labels_train })

In [6]:
data_test = pd.DataFrame({'test_text' : corpus_test, 'test_labels' : labels_test})

In [7]:
from bs4 import BeautifulSoup
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()
def remove_between_square_brackets(text):
    return re.sub('\\[[^]]*\\]', '', text)
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
data_train['train_text'] = data_train['train_text'].apply(denoise_text)
data_test['test_text'] = data_test['test_text'].apply(denoise_text)

C:\Users\burce\AppData\Local\Temp\ipykernel_21768\859156899.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [10]:
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-z\s]'
    text=re.sub(pattern,'',text)
    return text
data_train['train_text'] = data_train['train_text'].apply(remove_special_characters)
data_test['test_text'] = data_test['test_text'].apply(remove_special_characters)

In [11]:
def remove_punctuation(text):
    text = ''.join([char for char in text if char not in string.punctuation])
    return text
data_train['train_text'] = data_train['train_text'].apply(remove_punctuation)
data_test['test_text'] = data_test['test_text'].apply(remove_punctuation)

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\burce\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [14]:
def remove_stopwords(text, is_lower_case=False):
    tokenizer=ToktokTokenizer()
    stopword_list=nltk.corpus.stopwords.words('english')
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

data_train['train_text'] = data_train['train_text'].apply(remove_stopwords)
data_test['test_text'] = data_test['test_text'].apply(remove_stopwords)


In [15]:
stemmer = SnowballStemmer('english')
def stem_text(text):
    stemmer = SnowballStemmer('english')
    words = text.split()
    return ' '.join([stemmer.stem(word) for word in words])
data_train['train_text'] = data_train['train_text'].apply(stem_text)
data_test['test_text'] = data_test['test_text'].apply(stem_text)

In [16]:
data_train.to_csv('data_train.csv')

In [17]:
data_test.to_csv('data_test.csv')